# Calculation of Ground-State Energy of the Moscovium Atom by Quantum Algorithms
## Moscovium
Nuclear charge number $Z = 115$

Ground-state configuration [Rn] $5f^{14} 6d^{10} 7s^2 7p^3$

In the calculations we use 3 different basis sets constructed out of Dirac-Fock-Sturm radial orbitals [1]. 
The basis sets are presented in the Table, where $N_e$ is the number of active electrons, $N_{\rm orb}$  is the number of orbitals with the given total angular momentum projection, and $N_q$ is the number of required qubits.
The number of qubits always equals to $N_{\rm orb} - 2$ because of the parity mapping which allows to fix the parity of the state and the parity of the number of particles by dropping out two qubits


| Basis | <div style="width:290px">Radial Orbitals</div>             | $N_e$ | $N_{\rm orbs}$ | $N_q$ |
|   :- | :-:      |   :- | :- | :- |
|   $A$ |    $\{7s,7p_{1/2},7p_{3/2}\}$   |     5 | 8 | 6 |
|   $B$ |    $+\{8s, 8p_{1/2}, 8p_{3/2}\}$ |     5 | 16 | 14 |
|   $C$ |    $+\{6d_{3/2}, 6d_{5/2}\}\ \ \ \ \ $           |    15 | 26 | 24 |

## Variational Quantum Eigensolver (VQE)
Variational Quantum Eigensolver (VQE) is the hybrid quantum-classical algorithm [2] which constitutes in minimizing the expectation value

\begin{equation}
E_{\theta} =
\frac{ \left\langle \Psi(\theta) \left\vert H \right\vert \Psi(\theta) \right\rangle }
{ \left\langle \Psi(\theta) \vert \Psi(\theta) \right\rangle }
\end{equation}

Here the trial wave function is defined by

\begin{equation}
\left\vert \Psi(\theta) \right\rangle
= U\left(\theta\right) 
\left\vert \Psi_0 \right\rangle,
\end{equation}

with $\Psi_0$ standing for the initial state and $U(\theta)$ designating the parameterized unitary operator, which is called an "ansatz".
The preparation of the parameterized guess of the ground-state wave function and the measurement of the expectation value are performed on the quantum computer meanwhile the update of the parameters $\theta$ is performed on a classical one.
Here we use disentangled unitary coupled cluster [3] and hardware efficient [4] ansatzes, which are problem oriented and device specific, respectively.

Here we use Adam [5] and the Quantum Natural Gradients (QNG) [6, 7] approaches to perform multivariate optimization.

### Disentangled Unitary Coupled Cluster (dUCC) ansatz
In the dUCC ansatz, the initial guess of the ground-state wave function is given by

\begin{equation}
\left\vert \Psi_{\rm dUCC} \right\rangle =
\prod_i e^{t_{\mu_i} (\tau_{\mu_i} - \tau_{\mu_i}^\dagger)} \left\vert \Psi_0 \right\rangle,
\end{equation}

where $t_\mu$ is the cluster amplitude, the terms appear exactly once in the specific order [3], and

\begin{equation}
\hat{\tau}_\mu \equiv \hat{a}_{ij\dots}^{ab\dots} = \hat{a}_a^\dagger \hat{a}_b^\dagger \dots \hat{a}_j \hat{a}_i,
\end{equation}

where μ denotes indices i, j, . . . and a, b, . . . , which label occupied and virtual orbitals of the
reference state $\Psi_0$, respectively. 

In the present work, we restrict ourselves to the single and double excitations and refer to such ansatz as dUCC-SD. We arrange the single and double excitation operators in the order as they appear in the dUCC ansatz.

To transform each exponential in the product into the set of gates we utilize the combination of algorithms proposed in Refs [8-10] that allows to significantly reduce the number of required gates when compared ot the so-called greedy approach. 
The computer code realizing these algorimths is presented in the [GitHub](https://github.com/VAZaytsev/CVP).

Figure presents the difference $E_\theta − E_{\rm FCI}$ as a function of the number of iterations for basis
sets $A$, $B$, and $C$.
The calculations for the $C$ basis are still in progress.

<img src="graph/ducc_results.png" alt="drawing" width="600"/>

Table presents the number of parameters corresponding to the single $N_{\theta}^{(S)}$ and double $N_{\theta}^{(D)}$ excitations for $A$, $B$, and $C$ basis sets.
The size of the configuration space, i.e., the number of the Slater determinants with the odd parity and angular momentum projection $m = 1/2$, generated by the dUCC-SD ansatz $N_{\rm Sl.\ det.}^{\rm (SD)}$ is presented in the fourth column.
In the fifth column, the total size of the configuration space $N_{\rm Sl.\ det.}^{\rm (FCI)}$ for given quantum numbers is given.

| Basis | <div style="width:100px">$N_{\theta}^{(S)}$</div> | <div style="width:100px">$N_{\theta}^{(D)}$</div> | <div style="width:100px">$N_{\rm Sl.\ det.}^{\rm (SD)}$</div> | <div style="width:100px">$N_{\rm Sl.\ det.}^{\rm (FCI)}$</div> | <div style="width:290px">$E_{\theta} - E_{\rm FCI}$ [a.u.]</div> |
| :-: | --- |  --- | --- | --- | --- |
|  $A$ |  1 |   4 |       7 |       7 | $1.4 \times 10^{-6}$ |
|  $B$ |  9 |  68 |     450 |     472 | $2.4 \times 10^{-5}$ |
|  $C$ | 13 | 565 |  485820 |  500018 | $4.2 \times 10^{-5}$

### Hardware Efficient (HE) ansatz
Here we consider hardware efficient ansatzes, which differ by the structure of the layers and by the single-qubit rotation gates. Specifically, we use the ansatzes with two different entangling
layers being (i) merged and (ii) splitted by a layer of one-qubit rotations (see Figure).

| merged             |  splitted |
| :-: | :-: |
| <img src="graph/he_split_off.png" alt="drawing" width="450"/>  |  <img src="graph/he_split_on.png" alt="drawing" width="450"/> |

As single-qubit operations we utilize

\begin{equation}
U^{(1q)}_{zyz} = R_z(\theta_3)R_y(\theta_2)R_z(\theta_1),
\end{equation}

and

\begin{equation}
U^{(1q)}_{zx} = R_z(\theta_2)R_x(\theta_1),
\end{equation}

We present the results only for the smallest $A$ basis.
Since the initial values of $\theta$ parameters do not contain any physical meaning and chosen arbitrarily, the calculations for each particular ansatz were run for several times.
Here we choose the number of runs to be equal to the total number of parameters in the ansatz, divided by two.
Also, for each calculation we fix the maximal number of iteration equal to 1000.
Figure presents the dependence of the expectation value on the number of iterations for four different HE ansatzes with 5 layers.
Single-qubit rotations $U^{(1q)}_{zx}$ and $U^{(1q)}_{zyz}$ are used for the first and second rows of panels, respectively.
The left panels correspond to the ansatz with merged entangling layers, while the right panels are related to the splitted ones.

<img src="graph/he_results.png" alt="drawing" width="900"/>

## References
[1] I.I. Tupitsyn *et al.*, 
[Phys. Rev. A 68, 022511 (2003)](https://doi.org/10.1103/PhysRevA.68.022511); 
[Phys. Rev. A 72, 062503 (2005)](https://doi.org/10.1103/PhysRevA.72.062503);
[Phys. Rev. A 98, 022517 (2018)](https://doi.org/10.1103/PhysRevA.98.022517).

[2] A. Peruzzo *et al.*, 
[Nat. Commun. 5, 4213 (2014)](https://www.nature.com/articles/ncomms5213).

[3] F. A. Evangelista *et al.*, 
[J. Chem. Phys. 151, 244112 (2019)](https://doi.org/10.1063/1.5133059).

[4] A. Kandala *et al.*, 
[Nature 549, 242 (2017)](http://www.nature.com/doifinder/10.1038/nature23879).

[5] D.P. Kingma and J. Ba, 
[arXiv:1412.6980](https://arxiv.org/abs/1412.6980).

[6] J. Stokes *et al.*, 
[Quantum 4, 269 (2020)](https://quantum-journal.org/papers/q-2020-05-25-269/).

[7] D. Wierichs *et al.*, 
[Phys. Rev. Res. 2, 043246 (2020)](https://journals.aps.org/prresearch/abstract/10.1103/PhysRevResearch.2.043246).

[8] A. Cowtan *et al.*, 
[arXiv:2007.10515](https://arxiv.org/abs/2007.10515).

[9] V. Vandaele *et al.*, 
[arXiv:2104.00934](https://arxiv.org/abs/2104.00934).

[10] K.N. Patel *et al.*, 
[Quantum Inf. and Comp. 8, 0282 (2008)](http://dx.doi.org/10.26421/QIC8.3-4-4).